# 📊 Análisis Vitivinícola en San Rafael (Mendoza)

Este notebook realiza un análisis exploratorio y predictivo usando datos procesados desde Pentaho, que contienen información sobre superficie de viñedos, producción de uvas y rendimiento en San Rafael, Mendoza (2012–2017).

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [ ]:
# Cargar el dataset exportado desde Pentaho
file_path = 'resul.txt'
df = pd.read_csv(file_path, sep=';', decimal=',')
df.head()

In [ ]:
# Seleccionar columnas útiles y limpiar nombres
df = df[["anio", "localidad_nombre", "superficie_ha", "quintales_producidos", "rendimiento_kg_ha"]].copy()
df.columns = ["anio", "localidad", "superficie_ha", "quintales", "rendimiento"]

# Asegurar tipos numéricos
df["superficie_ha"] = pd.to_numeric(df["superficie_ha"], errors="coerce")
df["quintales"] = pd.to_numeric(df["quintales"], errors="coerce")
df["rendimiento"] = pd.to_numeric(df["rendimiento"], errors="coerce")

# Crear nueva columna: producción en kg
df["produccion_kg"] = df["quintales"] * 100

df.dropna(inplace=True)
df.head()

In [ ]:
# Agrupar por año (toda San Rafael)
df_anual = df.groupby("anio").agg({"superficie_ha": "sum", "produccion_kg": "sum"}).reset_index()
df_anual["rendimiento_kg_ha"] = df_anual["produccion_kg"] / df_anual["superficie_ha"]
df_anual

In [ ]:
# Visualización: evolución de producción y superficie
plt.figure(figsize=(10, 5))
plt.plot(df_anual["anio"], df_anual["produccion_kg"], marker='o', label='Producción (kg)')
plt.plot(df_anual["anio"], df_anual["superficie_ha"], marker='s', label='Superficie (ha)')
plt.title("Evolución de Producción y Superficie en San Rafael")
plt.xlabel("Año")
plt.legend()
plt.grid()
plt.show()

In [ ]:
# Evolución del rendimiento
plt.figure(figsize=(10,4))
plt.plot(df_anual["anio"], df_anual["rendimiento_kg_ha"], marker='d', color='green')
plt.title("Evolución del Rendimiento (kg/ha)")
plt.xlabel("Año")
plt.ylabel("kg/ha")
plt.grid()
plt.show()

In [ ]:
# Modelo predictivo: predecir producción en base a superficie y año
X = df_anual[["anio", "superficie_ha"]]
y = df_anual["produccion_kg"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)
model = LinearRegression().fit(X_train, y_train)

y_pred = model.predict(X_test)

print("R2 score:", r2_score(y_test, y_pred))
print("RMSE:", mean_squared_error(y_test, y_pred, squared=False))

In [ ]:
# Proyección para años futuros
future = pd.DataFrame({"anio": [2018, 2019, 2020], "superficie_ha": [df_anual["superficie_ha"].mean()] * 3})
future["produccion_pred"] = model.predict(future)
future